In [29]:
import openai
import json

In [30]:
with open("secrets.json", "r") as f:
    secrets = json.load(f)["key"]
    openai.api_key = secrets

In [31]:
model = "gpt-3.5-turbo"

In [49]:
transcription = """
I want to pick up my prescription, do my laundry, and get some groceries from the grocery store.
"""

messages = [
    {"role": "system", "content": "You are a climate-conscious schedule planner. Your client will give you a list of tasks to complete in a day. Output a schedule."},
    {"role": "user", "content": transcription},
]

functions = [
{
    "name": "generate_schedule",
    "description": "This function will generate a schedule for the user to view and follow.",
    "parameters": {
        "type": "object",
        "properties": {
            "events": {
                "type": "array",
                "description": "The list of events to be scheduled. Do not format.",
                "items": {
                "type": "object",
                "properties": {
                        "name_of_event": {
                            "type": "string",
                            "description": "The name of the event.",
                        },

                        "description_of_event": {
                            "type": "string",
                            "description": "The description of the event.",
                        },

                        "military_time_interval_of_event": {
                            "type": "string",
                            "description": "The time of the event in military time such as 20:00-21:00. Do not put AM or PM.",
                        },

                        "carbon_footprint_reduction_by_scheduling": {
                            "type": "string",
                            "description": "How the schedule reduces the carbon footprint.",
                        },
                    }
                },
            },
        },
    },
}
]

function_call = {
    "name": "generate_schedule",
}


In [41]:
completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions,
	function_call=function_call
)

print(completion)

{
  "id": "chatcmpl-81oO3Jgtu6y0eogvaEBqI6M0Wryao",
  "object": "chat.completion",
  "created": 1695442707,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "generate_schedule",
          "arguments": "{\n  \"events\": [\n    {\n      \"name_of_event\": \"Pick up prescription\",\n      \"description_of_event\": \"Go to the pharmacy and pick up prescription\",\n      \"military_time_interval_of_event\": \"10:00-10:30\",\n      \"carbon_footprint_reduction_by_scheduling\": \"Reduce carbon footprint by avoiding unnecessary trips\"\n    },\n    {\n      \"name_of_event\": \"Do laundry\",\n      \"description_of_event\": \"Do laundry at the laundromat\",\n      \"military_time_interval_of_event\": \"11:00-13:00\",\n      \"carbon_footprint_reduction_by_scheduling\": \"Reduce carbon footprint by doing laundry in bulk\"\n    },\n    {\n      \"name_of_ev

In [42]:
event_list = json.loads(completion.choices[0].message.function_call.arguments)['events']

print(f'The transcription """{transcription}""" returns: \n\n')

for event in event_list:
	print(f"The name of the event is {event['name_of_event']}")
	print(f"The description of the event is {event['description_of_event']}")
	print(f"The military time interval of the event is {event['military_time_interval_of_event']}")
	print()

The transcription """
I want to pick up my prescription, do my laundry, and get some groceries from the grocery store.
""" returns: 


The name of the event is Pick up prescription
The description of the event is Go to the pharmacy and pick up prescription
The military time interval of the event is 10:00-10:30
The carbon footprint reduction by scheduling is Reduce carbon footprint by avoiding unnecessary trips

The name of the event is Do laundry
The description of the event is Do laundry at the laundromat
The military time interval of the event is 11:00-13:00
The carbon footprint reduction by scheduling is Reduce carbon footprint by doing laundry in bulk

The name of the event is Get groceries
The description of the event is Go to the grocery store and buy groceries
The military time interval of the event is 14:00-15:00
The carbon footprint reduction by scheduling is Reduce carbon footprint by combining trips



In [55]:

# change the messages to feed the schedule back into the system and let ChatGPT generate a more optimal schedule
messages = [
    {"role": "system", "content": f"The user has the following schedule:\n\n{json.dumps(event_list, indent=4)}. Rewrite the user's schedule to lower the carbon footprint. Change the order of the events to accomplish this. Do not leave the ordering the same."},
]



In [56]:

completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions,
	function_call=function_call
)

print(completion)

{
  "id": "chatcmpl-81oVpVS9o9uXnLDwkqczuk74tsbFs",
  "object": "chat.completion",
  "created": 1695443189,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "generate_schedule",
          "arguments": "{\n  \"events\": [\n    {\n      \"name_of_event\": \"Pick up prescription\",\n      \"description_of_event\": \"Go to the pharmacy and pick up prescription\",\n      \"military_time_interval_of_event\": \"10:00-10:30\",\n      \"carbon_footprint_reduction_by_scheduling\": \"Reduce carbon footprint by avoiding unnecessary trips\"\n    },\n    {\n      \"name_of_event\": \"Do laundry\",\n      \"description_of_event\": \"Do laundry at the laundromat\",\n      \"military_time_interval_of_event\": \"11:00-13:00\",\n      \"carbon_footprint_reduction_by_scheduling\": \"Reduce carbon footprint by doing laundry in bulk\"\n    },\n    {\n      \"name_of_ev

In [58]:
event_list = json.loads(completion.choices[0].message.function_call.arguments)['events']

print(f'The transcription """{transcription}""" returns: \n\n')

for event in event_list:
	print(f"The name of the event is {event['name_of_event']}")
	print(f"The description of the event is {event['description_of_event']}")
	print(f"The military time interval of the event is {event['military_time_interval_of_event']}")
	print()

The transcription """
I want to pick up my prescription, do my laundry, and get some groceries from the grocery store.
""" returns: 


The name of the event is Pick up prescription
The description of the event is Go to the pharmacy and pick up prescription
The military time interval of the event is 10:00-10:30

The name of the event is Do laundry
The description of the event is Do laundry at the laundromat
The military time interval of the event is 11:00-13:00

The name of the event is Get groceries
The description of the event is Go to the grocery store and buy groceries
The military time interval of the event is 14:00-15:00

